##  주택청약 FAQ 시스템 챗봇 구현 - 문서 전처리 + RAG + Gradio ChatInterface

### 학습 목표

1. 텍스트 문서를 로드하고 구조화된 Q&A 쌍으로 파싱
2. LLM을 활용한 키워드 추출 및 요약 생성
3. Chroma 벡터 데이터베이스에 문서 임베딩 저장
4. MMR 검색 및 메타데이터 필터링 구현
5. RAG 체인 구성 및 문서 관련성 평가
6. Gradio를 사용한 대화형 챗봇 인터페이스 구현

---

### 사전 준비

**1. 환경 변수 설정**

`.env` 파일에 다음 내용을 추가하세요:
```
OPENAI_API_KEY=your-api-key-here
```

**2. 필수 패키지 설치**
```bash
pip install langchain-openai langchain-community langchain-core
pip install langchain-chroma chromadb
pip install python-dotenv gradio
```

**3. 데이터 파일**
- `data/housing_faq.txt` 파일 필요 (국토교통부 주택청약 FAQ 50개 Q&A)

---
### 추가 필기
- metadata 기반 필터링
	- 유사도 비교 + 필터링 검색
	- 필터를 걸었을 때 검색된 결과가 비어있는 경우 callback을 넣어줘서 대비한다.
	- 유사도 비교 함수 : as_retriever()
- 필터의 역할
	- 필터를 통해서 검색한 결과를 한번 걸러낸 후 Context에 저장하도록 한다.
	- 불필요한 검색을 하지 않으므로 환각을 줄인다.
	- 필터 종류
		- text로 sementic(의미 기반)으로 검색 할 때
		- 검색 범위 필터링 
			- <=, <, >, >= 
			- or, and
			-  

---

# 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

`(3) LLM 설정`

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4.1-mini',      # 사용할 모델
    temperature=0.1,            # 낮은 값: 일관된 답변 (0.0~2.0)
    top_p=0.9,                  # 토큰 샘플링 확률 임계값 (0.0~1.0)
)

/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# **문서 전처리 파이프라인**

* 문서 전처리의 첫 단계는 데이터 정제로, 원본 문서에서 불필요한 요소(HTML 태그, 특수문자, 중복 내용 등)를 제거하고 텍스트를 표준화하는 과정입니다. 이는 검색 품질과 직결되는 중요한 단계입니다.

* 문서 청킹(Chunking)은 긴 문서를 의미 있는 작은 단위로 분할하는 과정으로, 문장 단위나 단락 단위로 나누되 문맥의 연속성을 유지하는 것이 핵심입니다. 이는 검색 정확도와 답변 생성의 품질에 직접적인 영향을 미칩니다.

* 임베딩(Embedding) 생성은 텍스트를 고차원의 벡터로 변환하는 과정으로, 문서의 의미적 특성을 수치화하여 효율적인 검색을 가능하게 합니다. 이때 사용되는 임베딩 모델의 선택이 검색 성능을 좌우하는 중요한 요소가 됩니다.

* 마지막으로 벡터 데이터베이스 색인화 단계에서는 생성된 임베딩을 효율적으로 저장하고 검색할 수 있는 구조로 변환합니다. 이는 대규모 문서 집합에서도 빠른 검색을 가능하게 하는 핵심 요소입니다.


### 1) 문서 로드

- 국토교통부 주택청약 FAQ에서 일부 내용(청약자격, 청약통장)을 발췌하여 재가공
- Q1 ~ Q50까지 모두 50개의 문답이 포함된 텍스트 파일

In [5]:
# 파일 경로 설정
faq_text_file = "../data/housing_faq.txt"

# 파일 읽기 - 파이썬 내장 함수 사용
with open(faq_text_file, 'r') as f:
    faq_text = f.read()

# 파일 내용 확인
print(faq_text[:500])

Q1 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
A 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 
참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

Q2 해당 주택건설지역에 거주하고 있지 않다면 청약신청이 불가능한지?
A 해당 주택건설지역에 거주하고 있지 않더라도 청약가능지역에서 공급되는 주택에 청약신청이 가능하나, 같은 순위에서는 해당 주택건설지역의 거주자가 우선하여 주택을 공급받게 됩니다.
* 서울·인천·경기도 / 대전·세종·충남 / 충북 / 광주·전남 / 전북 / 대구·경북 / 부산·울산·경남 / 강원
다만, 수도권 대규모 택지개발지구 등에서 주택이 공급되는 경우 일정 비율의 


##### ***[실습] TextLoader를 사용하여, 텍스트 문서를 로드합니다.***

In [7]:
# 여기에 코드를 작성하세요.
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader

dir_loader = DirectoryLoader(
	path="../data",              # 파일 경로 - 현재 디렉토리
	glob=faq_text_file,     # 파일 확장자 - txt 파일만 로드
	loader_cls=TextLoader,  # TextLoader, CSVLoader, UnstructuredFileLoader 등 지원 
	show_progress=True,     # 진행 상태바 표시
)

dir_docs = dir_loader.load()

  0%|          | 0/1 [00:00<?, ?it/s]


In [8]:
# from langchain_community.document_loaders import TextLoader

# # TODO: TextLoader 클래스를 사용하여 FAQ 텍스트 파일을 로드
# loader = None
# docs = None
# len(docs)

from langchain_community.document_loaders import TextLoader

# TextLoader 클래스를 사용하여 FAQ 텍스트 파일을 로드
loader = TextLoader(faq_text_file)
docs = loader.load()
len(docs)

1

In [ ]:
# 문서 확인
print(docs[0].page_content[:500])

Q1 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
A 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 
참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

Q2 해당 주택건설지역에 거주하고 있지 않다면 청약신청이 불가능한지?
A 해당 주택건설지역에 거주하고 있지 않더라도 청약가능지역에서 공급되는 주택에 청약신청이 가능하나, 같은 순위에서는 해당 주택건설지역의 거주자가 우선하여 주택을 공급받게 됩니다.
* 서울·인천·경기도 / 대전·세종·충남 / 충북 / 광주·전남 / 전북 / 대구·경북 / 부산·울산·경남 / 강원
다만, 수도권 대규모 택지개발지구 등에서 주택이 공급되는 경우 일정 비율의 


In [10]:
# 문서 메타데이터 확인
docs[0].metadata

{'source': '../data/housing_faq.txt'}

### 2) 문서 전처리

`(1) 각 질문과 답변을 쌍으로 추출하여 정리 (정규표현식 활용)`

In [12]:
import re

def extract_qa_pairs(text):
    qa_pairs = []
    
    # 텍스트를 라인별로 분리하고 각 라인의 앞뒤 공백 제거
    lines = [line.strip() for line in text.split('\n')]
    current_question = None
    current_answer = []
    current_number = None
    in_answer = False
    
    for i, line in enumerate(lines):
        if not line:  # 빈 라인 처리
            if in_answer and current_answer and i + 1 < len(lines) and lines[i + 1].startswith('Q'):
                # 다음 질문이 시작되기 전 빈 줄이면 현재 QA 쌍 저장
                qa_pairs.append({
                    'number': current_number,
                    'question': current_question,
                    'answer': ' '.join(current_answer).strip()
                })
                in_answer = False
                current_answer = []
            continue
            
        # 새로운 질문 확인 (Q 다음에 숫자가 오는 패턴)
        q_match = re.match(r'Q(\d+)\s+(.*)', line)
        if q_match:
            # 이전 QA 쌍이 있으면 저장
            if current_question is not None and current_answer:
                qa_pairs.append({
                    'number': current_number,
                    'question': current_question,
                    'answer': ' '.join(current_answer).strip()
                })
            
            # 새로운 질문 시작
            current_number = int(q_match.group(1))
            current_question = q_match.group(2).strip().rstrip('?') + '?'  # 질문 마크 정규화
            current_answer = []
            in_answer = False
            
        # 답변 시작 확인
        elif line.startswith('A ') or (current_question and not current_answer and line):
            in_answer = True
            current_answer.append(line.lstrip('A '))
            
        # 기존 답변에 내용 추가
        elif current_question is not None and (in_answer or not line.startswith('Q')):
            if in_answer or (current_answer and not line.startswith('Q')):
                current_answer.append(line)
    
    # 마지막 QA 쌍 처리
    if current_question is not None and current_answer:
        qa_pairs.append({
            'number': current_number,
            'question': current_question,
            'answer': ' '.join(current_answer).strip()
        })
    
    # 번호 순서대로 정렬
    qa_pairs.sort(key=lambda x: x['number'])
    
    return qa_pairs

In [13]:
# QA 쌍 추출
qa_pairs = extract_qa_pairs(docs[0].page_content) 

print(f"추출된 QA 쌍 개수: {len(qa_pairs)}")
print(f"추출된 첫번째 QA: \n{qa_pairs[0]}")

추출된 QA 쌍 개수: 50
추출된 첫번째 QA: 
{'number': 1, 'question': '경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?', 'answer': '해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.'}


`(2) LLM으로 추가 정보를 추출`
- 텍스트에서 키워드와 핵심 개념을 추출하는 체인
- 메타데이터 or 본문(page_content)에 추가하여 검색에 활용    

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List

# 출력 형식 정의
class KeywordOutput(BaseModel):
    keyword: str = Field(description="텍스트에서 추출한 가장 중요한 키워드(법률용어, 주제 등))")
    summary: str = Field(description="텍스트의 간단한 요약")

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 텍스트 분석 전문가입니다. 
주어진 텍스트에서 중요한 키워드를 추출하고, 텍스트의 간단한 요약을 작성하는 것이 당신의 역할입니다.

## 추출 지침:
- 텍스트의 맥락을 고려하여 핵심 용어나 전문 용어를 추출합니다
- 주요 아이디어나 원리, 개념을 포함합니다
- 가장 중요한 키워드를 1개 추출합니다
- 요약은 1문장으로 간결하게 작성합니다

## 출력 형식:
- keyword: 가장 중요한 키워드 
- summary: 텍스트의 간단한 요약"""),
    
    ("user", "다음 텍스트를 분석해주세요:\n\n{input_text}")
])

# LCEL 체인 구성
llm_with_structure = llm.with_structured_output(KeywordOutput) 
keyword_extractor = prompt | llm_with_structure

# 텍스트 추출 테스트     
result = keyword_extractor.invoke(qa_pairs[0]['question']+qa_pairs[0]['answer'])
print("키워드:", result.keyword)
print("요약:", result.summary)

키워드: 해당 주택건설지역
요약: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역은 과천시 행정구역 전체를 의미한다.


`(3) QA 쌍을 문자열 포맷팅하고 문서 객체로 변환`

In [16]:
from langchain_core.documents import Document

def format_qa_pairs(qa_pairs):
    """
    추출된 QA 쌍을 포맷팅하여 문서 객체로 변환
    """
    processed_docs = []
    for pair in qa_pairs:

        # QA 쌍을 포맷팅
        formatted_output = (
            f"[{pair['number']}]\n"
            f"질문: {pair['question']}\n"
            f"답변: {pair['answer']}\n"
        )

        # 키워드와 요약 추출
        result = keyword_extractor.invoke(pair['question']+"\n\n"+pair['answer'])

        # 문서 객체 생성
        doc = Document(
            page_content=formatted_output,
            metadata={
                'question_id': int(pair['number']),
                'question': pair['question'],
                'answer': pair['answer'],
                'keyword': result.keyword,
                'summary': result.summary
            }
        )
        processed_docs.append(doc)

    return processed_docs


# QA 쌍 포맷팅
formatted_docs = format_qa_pairs(qa_pairs)
print(f"포맷팅된 문서 개수: {len(formatted_docs)}")

# 문서 확인
print(formatted_docs[0].page_content)
print("-" * 200)
# 문서 메타데이터 확인
pprint(formatted_docs[0].metadata)

/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='parsed', input_value=KeywordOutput(keyword='...체를 의미한다.'), input_type=KeywordOutput])
  return self.__pydantic_serializer__.to_python(
/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='parsed', input_value=KeywordOutput(keyword='...간에 포함된다.'), input_type=KeywordOutput])
  return self.__pydantic_serializer__.to_python(
/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializ

포맷팅된 문서 개수: 50
[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
{'answer': '해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 '
           '말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 '
           '주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 '
           '전역이 해당 주택건설지역에 해당됩니다.',
 'keyword': '해당 주택건설지역',
 'question': '경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?',
 'question_id': 1,
 'summary': '경기도 과천시에서 공급되는 주택의 해당 주택건설지역은 과천시 행정구역 전체를 의미한다.'}


/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='parsed', input_value=KeywordOutput(keyword='...격이 인정된다.'), input_type=KeywordOutput])
  return self.__pydantic_serializer__.to_python(


In [17]:
# 문서 저장
output_file = "../data/housing_faq_formatted.json"

with open(output_file, 'w', encoding='utf-8-sig') as f:
    json.dump([doc.model_dump() for doc in formatted_docs], f, indent=2, ensure_ascii=False)  # 한글이 유니코드로 변환되지 않도록 설정
print(f"포맷팅된 문서를 {output_file}에 저장했습니다.")

포맷팅된 문서를 ../data/housing_faq_formatted.json에 저장했습니다.


##### ***[실습] 문서 객체를 포맷팅하여 구성합니다.***

- 요약문을 시맨틱 검색에 활용합니다. 다음 구조로 문서 객체를 생성합니다. 
    - page_content: 요약
    - metadata: 기타 정보

In [ ]:
# 여기에 코드를 작성하세요.

In [ ]:
from langchain_core.documents import Document

def format_qa_pairs_with_summary(qa_pairs):
    """
    추출된 QA 쌍을 포맷팅하여 문서 객체로 변환
    """
    processed_docs = []
    for pair in qa_pairs:

        # 키워드와 요약 추출
        result = None

        # 문서 객체 생성
        doc = None

        processed_docs.append(doc)
        
    return processed_docs


In [19]:
# [강사]
from langchain_core.documents import Document

def format_qa_pairs_with_summary(qa_pairs):
    """
    추출된 QA 쌍을 포맷팅하여 문서 객체로 변환
    """
    processed_docs = []
    for pair in qa_pairs:

        # 키워드와 요약 추출
        result = keyword_extractor.invoke(pair['question']+"\n\n"+pair['answer'])

        # 문서 객체 생성
        doc = Document(
            page_content=result.summary,
            metadata={
                'question_id': int(pair['number']),
                'question': pair['question'],
                'answer': pair['answer'],
                'keyword': result.keyword,
            }
        )
        processed_docs.append(doc)

    return processed_docs

In [20]:
# QA 쌍 포맷팅
summary_formatted_docs = format_qa_pairs_with_summary(qa_pairs) 
print(f"포맷팅된 문서 개수: {len(summary_formatted_docs)}")

# 문서 확인
print(summary_formatted_docs[0].page_content)
print("-" * 200)
# 문서 메타데이터 확인
pprint(summary_formatted_docs[0].metadata)

/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='parsed', input_value=KeywordOutput(keyword='...간을 산정한다.'), input_type=KeywordOutput])
  return self.__pydantic_serializer__.to_python(
/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='parsed', input_value=KeywordOutput(keyword='...하여 판단한다.'), input_type=KeywordOutput])
  return self.__pydantic_serializer__.to_python(
/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializ

KeyboardInterrupt: 

In [ ]:
# 문서 저장
output_file = "../data/housing_faq_formatted_with_summary.json"

with open(output_file, 'w', encoding='utf-8-sig') as f:
    json.dump([doc.model_dump() for doc in summary_formatted_docs], f, indent=2, ensure_ascii=False)  # 한글이 유니코드로 변환되지 않도록 설정
print(f"포맷팅된 문서를 {output_file}에 저장했습니다.")

# 벡터 저장 

In [ ]:
# 문서 로드
from langchain_core.documents import Document

output_file = "../data/housing_faq_formatted.json"

with open(output_file, 'r', encoding='utf-8-sig') as f:
    formatted_docs = [Document(**doc) for doc in json.load(f)]

# 문서 확인
print(formatted_docs[0].page_content)

In [ ]:
print(formatted_docs[0].metadata)

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 문서 벡터 저장
vector_store = Chroma.from_documents(  
    documents=formatted_docs,
    embedding=embeddings,
    collection_name="housing_faq_db",
    persist_directory="../chroma_db",
)

In [ ]:
vector_store._collection.count()

In [ ]:
# vector_store.delete_collection()

##### ***[실습] 요약 문서(summary_formatted_docs)를 벡터 스토어에 저장합니다.*** 

- OpenAI (text-embedding-3-small) 임베딩 모델 사용
- Chroma DB 사용

**힌트**:
1. `Chroma.from_documents()` 메소드 사용
2. `embedding` 파라미터에 OpenAIEmbeddings 인스턴스 전달
3. `collection_name`과 `persist_directory` 지정
4. `summary_formatted_docs` 변수 사용

In [ ]:
# 여기에 코드를 작성하세요.


In [ ]:
# 문서 로드
from langchain_core.documents import Document

output_file = "../ata/housing_faq_formatted_with_summary.json"

with open(output_file, 'r', encoding='utf-8-sig') as f:
    summary_formatted_docs = [Document(**doc) for doc in json.load(f)]

# 문서 확인
print(summary_formatted_docs[0].page_content)

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 문서 벡터 저장
# vector_store_summary = None
vector_store_summary = Chroma.from_documents(  
    documents=formatted_docs,
    embedding=embeddings,
    collection_name="housing_faq_db",
    persist_directory="../chroma_db",
)

print(vector_store_summary._collection.count())

# 문서 검색

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 벡터 저장소 로드
vector_store = Chroma(
    collection_name="housing_faq_db",
    persist_directory="../chroma_db", 
    embedding_function=embeddings,
)

vector_store._collection.count()

##### ***[실습] 앞에서 저장한 요약문서 벡터 스토어를 로드합니다.*** 

- OpenAI (text-embedding-3-small) 임베딩 모델 사용
- Chroma DB 사용

**힌트**:
1. `Chroma()` 생성자 사용 (from_documents가 아님)
2. 저장 시 사용한 `collection_name`, `persist_directory` 동일하게 지정
3. `embedding_function` 파라미터에 OpenAIEmbeddings 전달

In [ ]:
# 여기에 코드를 작성하세요.


# 벡터 저장소 로드
# vector_store_summary = None
vector_store_summary = Chroma(
		collection_name="housing_faq_db",
		persist_directory="../chroma_db", 
		embedding_function=embeddings,
)

In [ ]:
# 검색기 생성 - 유사도 기반 상위 3개 문서 검색
retriever = vector_store.as_retriever(
    search_kwargs={"k": 3},
)

# 테스트 질문
query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

#### MMR (Maximal Marginal Relevance)

MMR은 검색 결과의 **관련성**과 **다양성**을 동시에 고려하는 알고리즘입니다.

**주요 파라미터**:
- `fetch_k`: 초기 검색 문서 수 (유사도 기준)
- `k`: 최종 반환 문서 수
- `lambda_mult`: 다양성 가중치
  - `0.0`: 최대 다양성 (서로 다른 문서 우선)
  - `1.0`: 최대 관련성 (유사도만 고려)
  - `0.5`: 균형 (관련성과 다양성을 동등하게)

##### ***[실습] MMR 검색기를 정의합니다.***  

- 문서 벡터 스토어를 사용
- 10개의 문서를 가져와서, 다양성 기반으로 3개를 선택 (다양성은 중간 수준 적용)

**힌트**:
1. `vector_store.as_retriever()` 메소드 사용
2. `search_type="mmr"` 파라미터 지정
3. `search_kwargs`에 `fetch_k=10`, `k=3`, `lambda_mult=0.5` 설정

In [ ]:
# 여기에 코드를 작성하세요.

mmr_retriever = None

# 테스트 질문
query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = mmr_retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print(result.metadata['question'])
    print(result.metadata['answer'])
    print("=" * 50)

### **[심화] 메타데이터 기반 필터링**

- Chroma 문서: https://docs.trychroma.com/docs/querying-collections/metadata-filtering

In [ ]:
# 단일 필드 정확히 일치
retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"keyword": "해당 주택건설지역"}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
# $eq 연산자 사용 - 정확히 일치 :: 유사도가 아닌 완전히 일치해야만 하는 경우


retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"keyword": {"$eq": "해당 주택건설지역"}}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
# $ne (Not Equal) 연산자 사용 - 정확히 일치하지 않는 문서 검색

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"keyword": {"$ne": "해당 주택건설지역"}}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
# $in 연산자로 여러 값 중 일치하는 문서 검색

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"keyword": {"$in": ["해당 주택건설지역", "청약예금"]}}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
# 숫자 범위 검색 ($gt, $gte, $lt, $lte) - question_id가 10 이상인 문서 검색

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"question_id": {"$gte": 10}}},
)

query = "무주택자 기준은 무엇인가요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
# $and로 여러 조건 조합 - keyword가 "주택건설지역"이고 question_id가 10 미만인 문서 검색

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"$and": [
        {"keyword": "해당 주택건설지역"}, 
        {"question_id": {"$lt": 10}}
    ]}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
# $or로 여러 조건 중 하나 일치하는 문서 검색 - keyword가 "주택건설지역"이거나 question_id가 10 이상인 문서 검색

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"$or": [
        {"keyword": "해당 주택건설지역"}, 
        {"question_id": {"$gte": 10}}
    ]}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
# 정규식 패턴 매칭 - page_content 본문에 "주택건설지역"이 포함된 문서 검색

retriever = vector_store.as_retriever(
    search_kwargs={'where_document': {'$contains': '해당 주택건설지역'}},
)

query = "수원시의 '해당 주택건설지역'은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

##### ***[실습] 메타데이터 필터링 조건을 적용하는 실습을 수행합니다..*** 

- 요약 문서 벡터 스토어 기반 MMR 검색기에 적용 

In [ ]:
# 여기에 코드를 작성하세요.

### 메타데이터 필터 LLM 추출

---
**추가필기**

### 메타데이터 필터 LLM 추출이란?
- 질문이 항상 바뀔 수 있는데 필터를 그때마다 하드코딩으로 바꿔줄 수 없으니 이것을 LLM으로 추출하는 방법이다.


---

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional, Literal

## Schema 상세 정의
class MetadataFilter(BaseModel):
    """Chroma DB 메타데이터 필터 조건"""

    # 키워드 필터
    keyword: Optional[str] = Field(default=None, description="검색할 키워드")
    keyword_operator: Optional[Literal["$eq", "$ne"]] = Field(
        default=None, description="키워드 비교 연산자"
    )

    # 질문 ID 범위 (하한)
    question_id_min: Optional[int] = Field(default=None, description="질문 ID 최소값")
    question_id_min_operator: Optional[Literal["$gt", "$gte"]] = Field(
        default=None, description="최소값 연산자 ($gt: 초과, $gte: 이상)"
    )

    # 질문 ID 범위 (상한)
    question_id_max: Optional[int] = Field(default=None, description="질문 ID 최대값")
    question_id_max_operator: Optional[Literal["$lt", "$lte"]] = Field(
        default=None, description="최대값 연산자 ($lt: 미만, $lte: 이하)"
    )

    # 논리 연산자
    logical_operator: Optional[Literal["$and", "$or"]] = Field(
        default="$and", description="복합 조건 결합 연산자"
    )

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 시스템 프롬프트
METADATA_FILTER_SYSTEM_PROMPT = """사용자 쿼리에서 Chroma DB 검색 필터 조건을 추출합니다.

## 추출 규칙

### 키워드 (keyword)
- 특정 단어/주제 검색 시 해당 키워드 추출
- keyword_operator: 일반적으로 "$eq" 사용

### 질문 ID 범위
- "N번 이상": question_id_min=N, question_id_min_operator="$gte"
- "N번 초과": question_id_min=N, question_id_min_operator="$gt"
- "N번 이하": question_id_max=N, question_id_max_operator="$lte"
- "N번 미만": question_id_max=N, question_id_max_operator="$lt"
- "N~M번 사이": 최소값과 최대값 모두 설정

### 논리 연산자
- 모든 조건 만족: "$and" (기본값)
- 하나라도 만족: "$or"

## 예시

1. 키워드만: "주택건설 관련 문서"
   → keyword="주택건설", keyword_operator="$eq"

2. ID 범위: "10번 이상 20번 이하"
   → question_id_min=10, question_id_min_operator="$gte",
     question_id_max=20, question_id_max_operator="$lte"

## 기간에 대해서 검색하는 것으로도 가능 - ex) 벤츠 SUV 몇 년식 이후 모델 검색
3. 복합 조건: "청약통장 관련 40~50번 문서"
   → keyword="청약통장", keyword_operator="$eq",
     question_id_min=40, question_id_min_operator="$gte",
     question_id_max=50, question_id_max_operator="$lte",
     logical_operator="$and"

해당 정보가 없으면 null 반환.
"""

# 메타데이터 추출 체인 구성
metadata_extraction_chain = (
    ChatPromptTemplate.from_messages([
        ("system", METADATA_FILTER_SYSTEM_PROMPT),
        ("human", "{query}")
    ])
    | llm.with_structured_output(MetadataFilter)
)

# 테스트: 필터 조건 추출
query = "'해당 주택건설지역' 관련 문서를 10번 이하인 문서중에서 검색해주세요"
filter_params = metadata_extraction_chain.invoke({"query": query})

print("추출된 필터 파라미터:")
print(filter_params.model_dump())

In [ ]:
def build_chroma_filter(filter_params: MetadataFilter) -> dict:
    """MetadataFilter를 Chroma DB 필터 딕셔너리로 변환
    
    Args:
        filter_params: MetadataFilter 인스턴스
        
    Returns:
        Chroma DB where 절에 사용할 필터 딕셔너리
    """
    conditions = []

    ### 필터 조건 구성

    # 키워드 조건
    if filter_params.keyword and filter_params.keyword_operator:
        conditions.append({
            "keyword": {filter_params.keyword_operator: filter_params.keyword}
        })

    # 질문 ID 최소값 조건
    if filter_params.question_id_min is not None and filter_params.question_id_min_operator:
        conditions.append({
            "question_id": {filter_params.question_id_min_operator: filter_params.question_id_min}
        })

    # 질문 ID 최대값 조건
    if filter_params.question_id_max is not None and filter_params.question_id_max_operator:
        conditions.append({
            "question_id": {filter_params.question_id_max_operator: filter_params.question_id_max}
        })

    # 조건 개수에 따른 필터 구성
    if len(conditions) == 0:
        return {}
    elif len(conditions) == 1:
        return conditions[0]
    else:
        logical_op = filter_params.logical_operator or "$and"
        return {logical_op: conditions}


# 테스트: 필터 딕셔너리 생성
filter_dict = build_chroma_filter(filter_params)
print("생성된 Chroma 필터:")
print(filter_dict)

In [ ]:
from langchain_core.runnables import chain


@chain
def metadata_filter_retriever(query: str):
    """메타데이터 필터를 추출하고 검색 수행
    
    Args:
        query: 사용자 검색 쿼리
        
    Returns:
        검색된 문서 리스트
    """
    # 1. 필터 조건 추출
    filter_params = metadata_extraction_chain.invoke({"query": query})

    # 2. Chroma 필터로 변환
    filter_dict = build_chroma_filter(filter_params)
    print(f"추출된 필터: {filter_dict}")

    # 3. 검색 실행
    retriever = vector_store.as_retriever(
        search_kwargs={"filter": filter_dict} if filter_dict else {}
    )
    return retriever.invoke(query)


# 테스트 실행
query = "청약통장 관련 문서를 40번과 50번 사이의 문서 중에서 검색해주세요"
results = metadata_filter_retriever.invoke(query)

print(f"\n검색된 문서 수: {len(results)}")
for i, result in enumerate(results, 1):
    print(f"\n--- 문서 {i} ---")
    print(f"내용: {result.page_content[:100]}...")
    print(f"키워드: {result.metadata.get('keyword', 'N/A')}")
    print(f"질문 ID: {result.metadata.get('question_id', 'N/A')}")

In [ ]:
# 다양한 쿼리 테스트 - 모순되지 않은 형태의 메세지로만 구성
test_queries = [
    "청약통장 관련 문서 찾아줘",        # 키워드만
    "질문 ID 10번 이상인 문서",         # ID 하한만
    "20번 이하 문서만 보여줘",          # ID 상한만
    "10번에서 30번 사이 문서",          # ID 범위
    "'해당 주택건설지역' 관련 10번 이하 문서",  # 복합
    "청약통장 관련 40~50번 문서",       # 복합 + 범위
]

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"쿼리: {query}")
    print('='*60)
    
    # 필터 추출 및 검색
    filter_params = metadata_extraction_chain.invoke({"query": query})
    filter_dict = build_chroma_filter(filter_params)
    print(f"필터: {filter_dict}")
    
    # 검색 실행
    results = metadata_filter_retriever.invoke(query)
    print(f"검색 결과: {len(results)}건")
    
    if results:
        print(f"첫 번째 문서 - 키워드: {results[0].metadata.get('keyword')}, ID: {results[0].metadata.get('question_id')}")

##### ***[실습] 메타데이터 필터링 체인의 성능을 개선합니다.*** 

- (예시)
    - 추가 예시 제공을 통해 필터링 적용 법위 확대

In [ ]:
# 여기에 코드를 작성하세요.

# RAG Chain

---
**추가필기**

### RAG로 실습진행 할 때 주의사항
- K값은 적게 하면 안된다. 왜냐?
	- 상대적 정밀도로 10, 20, 30, 50, 100, 200 등등으로 테스트해봐야한다.
	- 예를 들어 데이터가 100만개라고 했을 때 그 중 10개 검색되는 것과 200개 검색되는 것은 다르므로
- 정확한 검색을 위해
	- Vector store를 1, 2, 3으로 여러개 만들어서 검색
		-> keyword검색 후 hybrid처리 + graph기반 검색 가능
- 참조 없이 직접 답변 생성 시
	- 실제 검색을 한건지 확인을 위한 코드
		```python
		# 결과 출력
			print("답변:", result["response"]["answer"])
			print("\n참조 문서:")
			for i, doc in enumerate(result["source_documents"], 1):
					print(f"\n문서 {i}:")
					print(f"내용: {doc.page_content}")
		```
	- 해당 검색 결과에서도 관련성 있는 답변만 거르기 위해서 
		- 검색 문서 관련성 평가 해당 섹션 확인
		- 하지만 이런걸 llm으로 판단 하는 것도 정확하지 않다.
			- 이유
				1. 여러번 실행 시 답변이 달라진다.
			- 해결하기 위해서는
				1. 모델을 다른 것으로 사용해본다. > cloud, gemini - 모델이 갖는 지능의 차이가 있다.
				2. 여러 모델을 사용하여 하나로 만들거나
- FAQ의 경우에는
	- 임베딩한 문서와 원본을 다르게 전달하는 방식
	- 임베딩을 어떻게 chunk를 어떻게 구성하는게 중요하다.
	- 검색을 할 때 Q, A에서 Q로만 검색되도록 처리하는 방식으로 하면 검색의 정확도가 올라간다.
	- 원본에 대한 요약 (요약을 목차처럼 처리) -> 이것을 검색할 때 사용
- 원본을 요약하여 목차처럼 처리를 llm이 알아서 처리하도록하는게 정확할까? 아니면 사람이 하는게 정확할까?
	- Graph RAG가 이런 역할을 해준다.
	- 내용에서 LLM이 직접 관계성을 파악해서 추출 하는 방법이다.
	- chunking을 한 후 이 안에서 자세히 검색하는 방법
		- 확인 문서 : https://arxiv.org/pdf/2407.21059
- 차주 진행할 것 
	- 답변 생성 시 관련성 없는 것을 미리 걸러낸 후 답변이 나오도록 처리

---

### 1) 참조 문서 없이 직접 답변을 생성

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Prompt
template = '''Answer the question based only on the following context.

[Context]
{context}

[Question]
{question}

[Answer (in 한국어)]
'''

prompt = ChatPromptTemplate.from_template(template)


# 문서 포맷팅
def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])


# 검색기 생성 - 유사도 기반 상위 3개 문서 검색
retriever = vector_store.as_retriever(
    search_kwargs={"k": 3},
)


# Chain 구성
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Chain 실행
query = "수원시의 주택건설지역은 어디에 해당하나요?"
rag_chain.invoke(query)

### 2) 참조 문서를 답변과 함께 반환

`(1) 문서와 포맷팅된 컨텍스트를 함께 반환하는 함수`


In [ ]:
from typing import Dict

def get_context_and_docs(question: str) -> Dict:
    """문서와 포맷팅된 컨텍스트를 함께 반환
    
    Args:
        question: 검색할 질문

    Returns:
        Dict: 문서와 포맷팅된 컨텍스트, 검색된 문서 리스트
    """

    # 검색 결과 가져오기
    docs = retriever.invoke(question)
    return {
        "question": question,  # 질문
        "context": format_docs(docs),   # 문서 포맷팅된 컨텍스트
        "source_documents": docs   # 검색된 문서 리스트
    }

`(2) 컨텍스트와 질문을 입력으로 받아 답변을 생성하는 함수`

In [ ]:
from langchain_core.output_parsers import StrOutputParser

def prompt_and_generate_answer(input_data: Dict) -> Dict:
    """컨텍스트와 질문을 입력으로 받아 답변을 생성

    Args:
        input_data (Dict): 컨텍스트와 질문이 포함된 딕셔너리

    Returns:
        Dict: 생성된 답변과 소스 문서 정보가 포함된 딕셔너리
    """

    # LCEL 체인 구성 (StrOutputParser 사용)
    answer_chain = prompt | llm | StrOutputParser()
    answer = answer_chain.invoke({
        "question":input_data["question"],
        "context":input_data["context"]
    })

    return {
        "answer": answer,  # 생성된 답변 (answer_chain 결과)
        "source_documents": input_data["source_documents"]  # 소스 문서 정보 (input_data에서 가져옴)
    }

`(3) RAG 체인 구성`

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

# Chain 구성
rag_chain = (
    RunnableLambda(get_context_and_docs) |  # 문서와 컨텍스트 가져오기 
    {
        'response': RunnableLambda(prompt_and_generate_answer), # 답변 생성
		###  "source_documents": input_data["source_documents"] 코드로 question과 source_documents는 rag_chain에 전달 되므로 아래 코드는 불필요하다.
        # 'question': itemgetter("question"),  # 질문 그대로 전달
        # "source_documents": itemgetter("source_documents")   # 소스 반환
    }
)

In [ ]:
# Chain 실행
query = "수원시의 주택건설지역은 어디에 해당하나요?"
result = rag_chain.invoke(query)

# 결과 출력
print("답변:", result["response"]["answer"])
print("\n참조 문서:")
for i, doc in enumerate(result["source_documents"], 1):
    print(f"\n문서 {i}:")
    print(f"내용: {doc.page_content}")

### 3) 검색 문서 관련성 평가

`(1) 검색 문서와 질문 간의 관련성을 평가`


In [ ]:
# 검색 문서의 질문 관련성 평가

prompt = ChatPromptTemplate.from_messages([
    ("system", """주어진 컨텍스트가 질문에 답변하는데 필요한 정보를 포함하고 있는지 논리적으로 평가하세요.
단계적으로 진행하며, 평가결과에 대한 검증을 수행하세요.

다음 기준 중 하나 이상을 충족할 경우 'Yes'로 답변하고, 모두 충족하지 못하면 'No'로 답변하세요:

1. 컨텍스트가 질문에 답변하는데 필요한 정보를 직접적으로 포함하고 있는가?
2. 컨텍스트의 정보로부터 답변에 필요한 내용을 논리적으로 추론할 수 있는가?
3. 컨텍스트의 정보가 질문에 대한 답변을 제공할 수 있는가?

'Yes' 또는 'No'로만 답변하세요."""),
    ("human", """[컨텍스트]
{context}

[질문]
{question}""")
])

chain = prompt | llm | StrOutputParser()    # gpt-4.1-mini 모델 사용

for i, doc in enumerate(result["source_documents"], 1):
    print(f"\n문서 {i}:")
    print(f"내용: {doc.page_content}")
    relevance = chain.invoke({
        "context": doc.page_content,
        "question": query
    }).lower()

    print(f"평가 결과: {relevance}")

In [ ]:
# gpt-4.1 모델 사용

llm_gpt4o = ChatOpenAI(
    model='gpt-4.1',          # 사용할 모델
    temperature=0.1,          # 낮은 값: 일관된 답변 (0.0~2.0)
    top_p=0.9,                # 토큰 샘플링 확률 임계값 (0.0~1.0)
)

chain = prompt | llm_gpt4o | StrOutputParser()    # gpt-4.1 모델 사용

for i, doc in enumerate(result["source_documents"], 1):
    print(f"\n문서 {i}:")
    print(f"내용: {doc.page_content}")
    relevance = chain.invoke({
        "context": doc.page_content,
        "question": query
    }).lower()

    print(f"평가 결과: {relevance}")

##### ***[실습] 문서 관련성 평가 체인을 구조화 출력으로 구현합니다.*** 

- pydantic schema 사용
- with_structured_output 함수 사용

In [ ]:
# 여기에 코드를 작성하세요.

## Gradio 챗봇 인터페이스 (RAG 시스템 클래스 구현)

다음 클래스는 RAG 시스템의 전체 파이프라인을 캡슐화합니다.

**주요 메서드**:
- `_format_docs()`: 검색된 문서를 컨텍스트 문자열로 변환
- `_format_source_documents()`: 참조 문서를 사용자 친화적 형식으로 포맷
- `_evaluate_relevance()`: 검색된 문서의 질문 관련성 평가
- `_generate_answer()`: 컨텍스트 기반 답변 생성
- `generate_answer()`: Gradio 인터페이스용 메인 함수

**클래스 구조**:
1. LLM 초기화 (답변 생성용, 관련성 평가용)
2. 벡터 스토어 검색기 설정
3. 프롬프트 템플릿 정의
4. RAG 체인 구성

In [ ]:
import gradio as gr
from langchain_core.language_models import BaseChatModel
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from typing import List, Optional, Generator
from dataclasses import dataclass

@dataclass
class SearchResult:
    context: str
    source_documents: Optional[List]

class RAGSystem:
    def __init__(
            self, 
            llm: BaseChatModel, 
            eval_llm: BaseChatModel,
            retriever: VectorStoreRetriever
        ):
        if not llm:
            self.llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
        else:
            self.llm = llm

        if not eval_llm:
            self.eval_llm = ChatOpenAI(model="gpt-4.1", temperature=0)
        else:
            self.eval_llm = eval_llm

        if not retriever:
            raise ValueError("검색기(retriever)가 필요합니다.")
        else:
            self.retriever = retriever
        
    def _format_docs(self, docs: List) -> str:
        return "\n\n".join(doc.page_content for doc in docs)
    
    def _format_source_documents(self, docs: Optional[List]) -> str:
        if not docs:
            return "\n\nℹ️ 관련 문서를 찾을 수 없습니다."
        
        formatted_docs = []
        for i, doc in enumerate(docs, 1):
            metadata = doc.metadata if hasattr(doc, 'metadata') else {}
            source_info = []
            
            if 'question_id' in metadata:
                source_info.append(f"ID: {metadata['question_id']}")
            if 'keyword' in metadata:
                source_info.append(f"키워드: {metadata['keyword']}")
            if 'summary' in metadata:
                source_info.append(f"요약: {metadata['summary']}")
                
            formatted_docs.append(
                f"📚 참조 문서 {i}\n"
                f"• {' | '.join(source_info) if source_info else '출처 정보 없음'}\n"
                f"• 내용: {doc.page_content}"
            )
        
        return "\n\n" + "\n\n".join(formatted_docs)
    
    def _check_relevance(self, docs: List, question: str) -> List:
        """문서의 관련성 확인"""

        relevant_docs = []

        if not docs:
            return relevant_docs
            
        prompt = ChatPromptTemplate.from_messages([
            ("system", """주어진 컨텍스트가 질문에 답변하는데 필요한 정보를 포함하고 있는지 평가하세요.

        다음 기준 중 하나 이상을 충족할 경우 'Yes'로 답변하고, 모두 충족하지 못하면 'No'로 답변하세요:

        1. 컨텍스트가 질문에 답변하는데 필요한 정보를 직접적으로 포함하고 있는가?
        2. 컨텍스트의 정보로부터 답변에 필요한 내용을 논리적으로 추론할 수 있는가?

        'Yes' 또는 'No'로만 답변하세요."""),
            ("human", """[컨텍스트]
        {context}

        [질문]
        {question}""")
        ])
        
        chain = prompt | self.eval_llm | StrOutputParser()

        for doc in docs:
            result = chain.invoke({
                "context": doc.page_content,
                "question": question
            }).lower()

            print(f"문서 {doc.metadata['question_id']} 관련성 확인 결과: {result}")
            print(f"문서 {doc.metadata['question_id']} 내용:")
            print(doc.page_content)
            print("-" * 50)

            if "yes" in result:
                relevant_docs.append(doc)
            
        return relevant_docs
    
    def search_documents(self, question: str) -> SearchResult:
        try:
            docs = retriever.invoke(question)
            print(f"검색된 문서 개수: {len(docs)}")
            relevant_docs = self._check_relevance(docs, question) 
            print(f"관련 문서 개수: {len(relevant_docs)}")
            
            return SearchResult(
                context=self._format_docs(relevant_docs) if relevant_docs else "관련 문서를 찾을 수 없습니다.",
                source_documents=relevant_docs,
            )
        except Exception as e:
            print(f"문서 검색 중 오류 발생: {e}")
            return SearchResult(
                context="문서 검색 중 오류가 발생했습니다.",
                source_documents=None,
            )
    
    def generate_answer(self, message: str, history: List) -> Generator[str, None, None]:
            """Gradio 스트리밍 출력을 위한 제너레이터 함수"""
            
            # 1. 문서 검색 
            search_result = self.search_documents(message)
            
            if not search_result.source_documents:
                yield "죄송합니다. 관련 문서를 찾을 수 없어 답변하기 어렵습니다. 다른 질문을 해주시겠습니까?"
                return
                        
            # 2. 프롬프트 템플릿 설정
            prompt = ChatPromptTemplate.from_messages([
                ("system", """다음 지침을 따라 질문에 답변해주세요:
                1. 주어진 문서의 내용만을 기반으로 답변하세요.
                2. 문서에 명확한 근거가 없는 내용은 "근거 없음"이라고 답변하세요.
                3. 답변하기 어려운 질문은 "잘 모르겠습니다"라고 답변하세요.
                4. 추측이나 일반적인 지식을 사용하지 마세요."""),
                ("human", "문서들:\n{context}\n\n질문: {question}")
            ])
            
            # 3. RAG Chain 구성
            chain = prompt | self.llm | StrOutputParser()
            
            full_answer = ""
            try:
                # 4. 스트리밍 실행 (chain.stream 사용)
                for chunk in chain.stream({
                    "context": search_result.context,
                    "question": message
                }):
                    full_answer += chunk
                    # 현재까지 생성된 텍스트를 Gradio UI에 즉시 반영
                    yield full_answer
                
                # 5. 답변 생성이 완료된 후 참조 문서 추가
                sources = self._format_source_documents(search_result.source_documents)
                final_response = f"{full_answer}\n\n---\n{sources}"
                yield final_response
                
            except Exception as e:
                yield f"답변 생성 중 오류가 발생했습니다: {str(e)}"

# Gradio 인터페이스 설정

rag_system = RAGSystem(
    llm=ChatOpenAI(model="gpt-4.1-nano", temperature=0),  # 답변 생성에 사용할 모델
    eval_llm=ChatOpenAI(model="gpt-4.1-mini", temperature=0), # 문서 관련성 평가에 사용할 모델
    retriever=vector_store.as_retriever(search_kwargs={"k": 3})
)

demo = gr.ChatInterface(
    fn=rag_system.generate_answer,
    title="RAG QA 시스템",
    description="""
    질문을 입력하면 관련 문서를 검색하여 답변을 생성합니다.
    모든 답변에는 참조한 문서의 출처가 표시됩니다.
    """,
    examples=[
        ["수원시의 주택건설지역은 어디에 해당하나요?"],
        ["무주택 세대에 대해서 설명해주세요."],
        ["2순위로 당첨된 사람이 청약통장을 다시 사용할 수 있나요?"],
    ],
)

# 데모 실행
demo.launch()

In [ ]:
# Gradio 인터페이스 종료
demo.close()

---

# **[실습] 주택청약 FAQ 시스템 구현**

### **문제 설명**
이전 코드를 기반으로 주택청약 FAQ 시스템을 다음 요구사항에 맞춰 개선합니다. 

1. 응답 품질 향상 (1개 이상)
   - 생성된 답변의 품질을 평가 (답변이 불충분한 경우 예외 처리)
   - 관련성 높은 FAQ 문서 검색 (임베딩 모델, 청크 크기, 벡터 검색 방법 등)

2. 사용자 경험 개선 (1개 이상)
   - 대화 이력 관리 기능 추가 (요약, 트리밍 기능 등 고려)
   - 최근 대화 기반 컨텍스트 구성 
   - 사용자 프로필 기반 맞춤 응답

### **제약 조건**
- Gradio ChatInterface 사용
- RAG 구조 유지

In [ ]:
# 여기에 코드를 작성하세요. 